<a href="https://colab.research.google.com/github/deltorobarba/chemistry/blob/main/quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Computing for Ground-state energy estimation**

In [2]:
!pip install pyscf matplotlib pennylane -q

In [3]:
# Hartree-Fock SCF for Ground state energy - As benchmark
from pyscf import gto, scf
mol = gto.M(
    atom = '''
    O  0.000000  0.000000  0.000000
    H  0.000000  0.757000  0.586000
    H  0.000000 -0.757000  0.586000
''',
    basis = 'cc-pvdz',
)
mf = scf.RHF(mol)
conv_data = []
def callback(envs):
    conv_data.append(envs['e_tot'])
mf.callback = callback
hf_scf_ground_state_energy = mf.kernel()

converged SCF energy = -76.0267936449738


In [4]:
# Variational Quantum Eigensolver for Ground-state energy estimation
import pennylane as qml
from pennylane import numpy as np

# Define the molecular system
symbols = ['O', 'H', 'H']
# The coordinates should be provided in atomic units (Bohr)
coordinates = np.array([0.0, 0.0, 0.0, 1.432, 1.107, 0.0, -1.432, 1.107, 0.0])  # Approximate atomic units

# Generate the molecular Hamiltonian using PennyLane's quantum chemistry module
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,  # Singlet state
    basis='sto-3g',  # Simple minimal basis set
    active_electrons=4,  # 2 electrons from hydrogen and 2 from oxygen (for a minimal example)
    active_orbitals=4,  # Adjust as needed for accuracy
    mapping='jordan_wigner'  # Fermion-to-qubit mapping
)

# Define the quantum device
dev = qml.device('default.qubit', wires=qubits)

# Define the ansatz (parameterized quantum circuit)
def ansatz(params):
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i + 1])

# Define the cost function (expectation value of the Hamiltonian)
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params)
    return qml.expval(H)

# Perform the VQE optimization
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
params = np.random.randn(qubits)
for i in range(100):
    params = optimizer.step(cost_fn, params)

# Extract the ground state energy
vqe_ground_state_energy = cost_fn(params)

print("Ground state energy of H2O molecule (Pennylane VQE):", vqe_ground_state_energy)

Ground state energy of H2O molecule (Pennylane VQE): -74.48444328875853


In this example I use 8 qubits for the water molecule. This is determined by 4 active molecular orbitals to describe the electronic structure of the water molecule (`active_orbitals=4`), where each orbital has two spin states (two qubits per orbital) under Jordan-Wigner transformation. Fermion-to-qubit mapping for each spin-orbital to one qubit. For each active orbital, there are two spin states (spin-up and spin-down)

In [5]:
# Quantum Computing for Ground-state energy estimation using QPE
import pennylane as qml
from pennylane import numpy as np
from scipy.linalg import expm

# Define molecular system
symbols = ['O', 'H', 'H']
coordinates = np.array([0.0, 0.0, 0.0, 1.432, 1.107, 0.0, -1.432, 1.107, 0.0])

# Generate the molecular Hamiltonian (PennyLane quantum chemistry module)
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,
    basis='sto-3g',
    active_electrons=4,
    active_orbitals=4,
    mapping='jordan_wigner'
)

# Get the wire ordering for the qubits representing the molecule
molecule_wires = list(range(qubits))

# Convert the Hamiltonian to a sparse matrix representation
H_matrix = H.sparse_matrix(wire_order=molecule_wires).toarray()

# Define the quantum device
n_auxiliary_qubits = 4  # Number of qubits for the phase estimation register
total_wires = n_auxiliary_qubits + qubits
dev = qml.device('default.qubit', wires=total_wires)

# Function for controlled unitary evolution using the Hamiltonian
def controlled_unitary(time, wires):
    # Time evolution operator e^(-iHt), where H is the Hamiltonian
    U = expm(-1j * H_matrix * time)

    # Apply the unitary controlled on an auxiliary qubit
    qml.ControlledQubitUnitary(U, control_wires=wires[0], wires=wires[1:])

# Quantum Phase Estimation (QPE) circuit
def qpe_circuit(time):
    # Step 1: Prepare auxiliary qubits in a superposition state (Hadamard transform)
    for i in range(n_auxiliary_qubits):
        qml.Hadamard(wires=i)

    # Step 2: Apply controlled unitary operations (controlled e^(-iHt))
    for i in range(n_auxiliary_qubits):
        # Calculate the time for this iteration
        t = 2 ** i * time
        # Apply the controlled unitary with appropriate wires
        control_wire = i
        target_wires = list(range(n_auxiliary_qubits, total_wires))
        wires = [control_wire] + target_wires
        controlled_unitary(t, wires=wires)

    # Step 3: Inverse Quantum Fourier Transform (QFT)
    qml.adjoint(qml.QFT)(wires=range(n_auxiliary_qubits))

# Define ansatz
def ansatz(params):
    for i in range(qubits):
        qml.RY(params[i], wires=n_auxiliary_qubits + i)
    for i in range(qubits - 1):
        qml.CNOT(wires=[n_auxiliary_qubits + i, n_auxiliary_qubits + i + 1])

# Define the QPE circuit combined with the ansatz
@qml.qnode(dev)
def phase_estimation(params):
    # Prepare the ground state using the VQE ansatz
    ansatz(params[1:])

    # Apply the QPE circuit
    qpe_circuit(params[0])

    # Measure the auxiliary qubits in the computational basis
    return qml.probs(wires=range(n_auxiliary_qubits))

# Parameters: [time, ansatz_params]
params = np.array([0.1] + list(np.random.randn(qubits)))

# Run the phase estimation
probs = phase_estimation(params)

# The measured probabilities correspond to the binary representation of the eigenvalues
# Convert these to eigenphases and then to energy estimates

# Helper function to convert binary to decimal and to energy
def binary_to_energy(binary_probabilities, time):
    # Get the most probable state (highest probability)
    max_prob_idx = np.argmax(binary_probabilities)

    # Convert index to binary (phase estimate)
    binary_phase = format(max_prob_idx, '0' + str(n_auxiliary_qubits) + 'b')

    # Convert binary phase to decimal
    phase_estimate = sum([int(bit) * 2 ** -(i + 1) for i, bit in enumerate(binary_phase)])

    # Calculate the energy from the phase
    energy_estimate = 2 * np.pi * phase_estimate / time

    return energy_estimate

# Extract the ground state energy
qpe_ground_state_energy = binary_to_energy(probs, params[0])

print("Ground state energy of H2O molecule (Pennylane QPE):", qpe_ground_state_energy)

Ground state energy of H2O molecule (Pennylane QPE): 23.561944901923447


[IonQ: Ground-state energy estimation of the water molecule on a trapped ion quantum computer](https://ionq.com/links/Water_Molecule_VQE_Simulation.pdf)

https://pennylane.ai/qml/demos/tutorial_quantum_chemistry/

In [7]:
print("Ground state energy of H2O molecule (Hartree-Fock SCF):", hf_scf_ground_state_energy)
print("Ground state energy of H2O molecule (Variational Quantum Eigensolver):", vqe_ground_state_energy)
print("Ground state energy of H2O molecule (Quantum Phase Estimation):", qpe_ground_state_energy)

Ground state energy of H2O molecule (Hartree-Fock SCF): -76.02679364497384
Ground state energy of H2O molecule (Variational Quantum Eigensolver): -74.48444328875853
Ground state energy of H2O molecule (Quantum Phase Estimation): 23.561944901923447
